In [57]:
# I copied and pasted the table from Wikipedia into a CSV file and imported it into IBM Cloud Pak for Data.
# Then I chose "insert to code" for the CSV file, which returned the following code (until df_data_1 = pd.read_csv(body))

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# Message from IBM (pasted when calling the "insert to code" function. I replaced my credentials with "CLIENT_ID")
# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_CLIENT_ID = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-sr-LvN8cjMcjxJij-z8sHBcc0zK-6CY3Q2Kvo0xlfAy',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_CLIENT_ID.get_object(Bucket='datasciencefinalassignmentmodule8-donotdelete-pr-p7xhaymzrujbuv',Key='Neighborhoods.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)

# dropping null value columns to avoid errors 
df_data_1.dropna(inplace = True) 
  
# new data frame with split value columns 
new = df_data_1["PostalCode;Borough;Neighbourhood"].str.split(";", n = 2, expand = True) 
  
# making separate first name column from new data frame 
df_data_1["PostalCode"]= new[0] 
df_data_1["Borough"]= new[1]
df_data_1["Neighborhood"]= new[2]

# Dropping old Name columns 
df_data_1.drop(columns =["PostalCode;Borough;Neighbourhood"], inplace = True)

# Get names of indexes for which column Borough and Neighborhood are "Not assigned"
indexNames = df_data_1[df_data_1['Borough'] == 'Not assigned' ].index

# Delete these row indexes from dataFrame
df_data_1.drop(indexNames , inplace=True)

# Reset the index and clean the data
df_data_1 = df_data_1.reset_index(drop=True)
df_data_1['Neighborhood'] = df_data_1['Neighborhood'].str.replace(r'"', '')
df_data_1['Neighborhood'] = df_data_1['Neighborhood'].str.replace(r';', ',')

# df display 
df_data_1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [58]:
df_data_1.shape

(103, 3)